In [ ]:
import pandas as pd

dados_treino = pd.read_csv("/content/drive/MyDrive/Alura/Curso_word2vec-embedding/treino.csv")
dados_teste = pd.read_csv("/content/drive/MyDrive/Alura/Curso_word2vec-embedding/teste.csv")

dados_treino.head(5) # Mostra os primeiros 5

,title,text,date,category,subcategory,link
0,"Após polêmica, Marine Le Pen diz que abomina n...",A candidata da direita nacionalista à Presidên...,2017-04-28,mundo,NaN,http://www1.folha.uol.com.br/mundo/2017/04/187...
1,"Macron e Le Pen vão ao 2º turno na França, em ...",O centrista independente Emmanuel Macron e a d...,2017-04-23,mundo,NaN,http://www1.folha.uol.com.br/mundo/2017/04/187...
2,"Apesar de larga vitória nas legislativas, Macr...",As eleições legislativas deste domingo (19) na...,2017-06-19,mundo,NaN,http://www1.folha.uol.com.br/mundo/2017/06/189...
3,"Governo antecipa balanço, e Alckmin anuncia qu...",O número de ocorrências de homicídios dolosos ...,2015-07-24,cotidiano,NaN,http://www1.folha.uol.com.br/cotidiano/2015/07...
4,"Após queda em maio, a atividade econômica sobe...","A economia cresceu 0,25% no segundo trimestre,...",2017-08-17,mercado,NaN,http://www1.folha.uol.com.br/mercado/2017/08/1...


In [ ]:
dados_treino.sample(5) # Mostra 5 linhas aleatórias 

,title,text,date,category,subcategory,link
19444,\n\t\tNão faltou dinheiro para começar o plano...,Opinião JOSÉ HENRIQUE MARIANTE ENVIADO ESPE...,2016-05-08,esporte,olimpiada-no-rio,http://www1.folha.uol.com.br/esporte/olimpiada...
75264,Jobson é suspenso pela Fifa; Botafogo tenta re...,O atacante Jobson enfrenta um novo problema fo...,2015-04-24,esporte,NaN,http://www1.folha.uol.com.br/esporte/2015/04/1...
51829,Arquivo-bomba da Odebrecht chega às mãos de Moro,Um dos maiores mistérios que cercam a delação ...,2017-08-25,colunas,monicabergamo,http://www1.folha.uol.com.br/colunas/monicaber...
12168,Santos vence São Paulo e vai à 7ª final seguid...,O Santos chegou à final do Campeonato Paulista...,2015-04-19,esporte,NaN,http://www1.folha.uol.com.br/esporte/2015/04/1...
1629,"Roubada e apreendida pela polícia, arma volta ...","Duas vezes desviado, duas vezes nas mãos do cr...",2015-05-24,cotidiano,NaN,http://www1.folha.uol.com.br/cotidiano/2015/05...


In [ ]:
# Ao inves do nltk, será utilizado o spaCy, utilizada para processamento avançado de linguagem natural
# É necessário instalar a linguagem português

!python -m spacy download pt_core_news_sm #Rodar apenas uma vez

✔ Download and installation successful
You can now load the model via spacy.load('pt_core_news_sm')


In [ ]:
import spacy
nlp = spacy.load("pt_core_news_sm")

In [ ]:
texto = "Hoje, é um novo dia, de um novo tempo,, que começou"

doc = nlp(texto)

In [ ]:
type(doc)

spacy.tokens.doc.Doc

In [ ]:
# text -> nlp(tokenizer -> tagger -> parser -> ner -> ...) -> Doc

In [ ]:
for i in range(0, len(doc)):
  print(doc[i])

Hoje
,
é
um
novo
dia
,
de
um
novo
tempo
,
,
que
começou


In [ ]:
doc.ents # Entidades, nomes, cidades, substantivos

()

In [ ]:
doc[2].is_stop # é stopwords ?

True

In [ ]:
textos_para_tratamento = (titulo.lower() for titulo in dados_treino['title'])
# Como é um gerador, só vai iterar sobre determinado elemento quando esse elemento for chamado

In [ ]:
# Para ver como ficam os elementos, basta listá-los
# list(textos_para_tratamento)

In [ ]:
def trata_textos(doc):
  tokens_validos = []
  for token in doc:
    valido = not token.is_stop and token.is_alpha # Estou descartando as stopwords e símbolos e coletando apenas as palavras
    if valido:
      tokens_validos.append(token.text) # coletando não o token mas sim seu text
  
  if len(tokens_validos) > 2: # para fazer CBOW preciso de pelo menos 3 palavras para dar contexto
    return " ".join(tokens_validos)

In [ ]:
trata_textos(doc)

'Hoje dia começou'

In [ ]:
from time import time 
t0 = time()
textos_tratados = [trata_textos(doc) for doc in nlp.pipe(textos_para_tratamento, batch_size=1000, n_process = -1)]
# nlp.pipe é para passar muitos docs de uma vez, 
# batch_size são de quantos em quantos (temos 90000)
# n_process é o número de núcleos no processador para paralelizar 


t1 = time() - t0
print(t1/60)

62.99237972895305


In [ ]:
titulos_tratados = pd.DataFrame({"titulo": textos_tratados})
titulos_tratados.head(5)

,titulo
0,polêmica marine le pen abomina negacionistas h...
1,macron e le pen a o turno frança revés siglas ...
2,apesar larga vitória legislativas macron terá ...
3,governo antecipa balanço e alckmin anuncia que...
4,queda maio a atividade econômica sobe junho bc


In [ ]:
print("sem eliminar os nulls temos ", len(titulos_tratados))
titulos_tratados = titulos_tratados.dropna().drop_duplicates()
print("eliminando os nulls restam ", len(titulos_tratados))

sem eliminar os nulls temos  90000
eliminando os nulls restam  86113


In [ ]:
# Fazer uma lista onde cada elemento é uma lista de palavras associadas a cada um dos nossos títulos
lista_lista_tokens = [titulo.split(" ") for titulo in titulos_tratados.titulo]

In [ ]:
# Para mostrar o processo de log utilizamos:

import logging 
logging.basicConfig(format='%(asctime)s : - %(message)s', level = logging.INFO)


from gensim.models import Word2Vec
w2v_modelo = Word2Vec(sg = 0, # skipgram = 0 => cbow como arquitetura
                      window = 2, # quantidade de palavras ao redor pelo contexto
                      size = 300, # quantidade de dimensões do vetor
                      min_count = 5, # palavras que se repetem pelo menos 5 vezes são contadas
                      alpha = 0.03, # taxa de aprendizagem
                      min_alpha = 0.007) # menor valor possível para a taxa de aprendizagem

#Construindo o vocabulario do modelo
w2v_modelo.build_vocab(lista_lista_tokens, progress_per=1000000)

2021-05-29 03:59:22,818 : - 'pattern' package not found; tag filters are not available for English
2021-05-29 03:59:22,830 : - collecting all words and their counts
2021-05-29 03:59:22,832 : - PROGRESS: at sentence #0, processed 0 words, keeping 0 word types
2021-05-29 03:59:23,050 : - collected 39968 word types from a corpus of 597929 raw words and 86113 sentences
2021-05-29 03:59:23,051 : - Loading a fresh vocabulary
2021-05-29 03:59:23,101 : - effective_min_count=5 retains 13006 unique words (32% of original 39968, drops 26962)
2021-05-29 03:59:23,103 : - effective_min_count=5 leaves 552614 word corpus (92% of original 597929, drops 45315)
2021-05-29 03:59:23,159 : - deleting the raw counts dictionary of 39968 items
2021-05-29 03:59:23,161 : - sample=0.001 downsamples 11 most-common words
2021-05-29 03:59:23,163 : - downsampling leaves estimated 502900 word corpus (91.0% of prior 552614)
2021-05-29 03:59:23,229 : - estimated required memory for 13006 words and 300 dimensions: 377174

In [ ]:
dir(w2v_modelo) # todos os métodos do modelo

['__class__',
 '__contains__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__getitem__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 '_adapt_by_suffix',
 '_check_input_data_sanity',
 '_check_training_sanity',
 '_clear_post_train',
 '_do_train_epoch',
 '_do_train_job',
 '_get_job_params',
 '_get_thread_working_mem',
 '_job_producer',
 '_load_specials',
 '_log_epoch_end',
 '_log_epoch_progress',
 '_log_progress',
 '_log_train_end',
 '_minimize_model',
 '_raw_word_count',
 '_save_specials',
 '_set_train_params',
 '_smart_save',
 '_train_epoch',
 '_train_epoch_corpusfile',
 '_update_job_params',
 '_worker_loop',
 '_worker_loop_corpusfile',
 'accuracy',
 'alpha',
 'batch_words',
 'build_vocab',
 'build_vocab_from_freq',
 'ca

In [ ]:
# Quantidade de amostras que utilizamos para  construir nosso vocabulário
w2v_modelo.corpus_count

86113

In [ ]:
# Colocando o modelo para o treinamento

w2v_modelo.train(lista_lista_tokens,
                 total_examples = w2v_modelo.corpus_count,
                 epochs = 30)

2021-05-29 03:59:25,873 : - training model with 3 workers on 13006 vocabulary and 300 features, using sg=0 hs=0 sample=0.001 negative=5 window=2
2021-05-29 03:59:26,890 : - EPOCH 1 - PROGRESS: at 61.95% examples, 310747 words/s, in_qsize 5, out_qsize 0
2021-05-29 03:59:27,396 : - worker thread finished; awaiting finish of 2 more threads
2021-05-29 03:59:27,413 : - worker thread finished; awaiting finish of 1 more threads
2021-05-29 03:59:27,432 : - worker thread finished; awaiting finish of 0 more threads
2021-05-29 03:59:27,434 : - EPOCH - 1 : training on 597929 raw words (503013 effective words) took 1.5s, 325475 effective words/s
2021-05-29 03:59:28,467 : - EPOCH 2 - PROGRESS: at 65.27% examples, 323714 words/s, in_qsize 5, out_qsize 0
2021-05-29 03:59:28,925 : - worker thread finished; awaiting finish of 2 more threads
2021-05-29 03:59:28,942 : - worker thread finished; awaiting finish of 1 more threads
2021-05-29 03:59:28,958 : - worker thread finished; awaiting finish of 0 more t

(15086746, 17937870)

In [ ]:
w2v_modelo.wv.most_similar('google')

In [ ]:
# Agora, realizando o treinamento do modelo com Skip-Gram

w2v_modelo_sg = Word2Vec(sg = 1, # Utilizando a arquitetura Skip-Gram
                         window = 5, # Quantidade de palavras ao redor para o contexto (nesse caso precisa ser maior dada a arquitetura)
                         min_count = 5, # Contar apenas palavras que se repetem pelo menos 5 vezes
                         alpha = 0.03, # Taxa de aprendizagem 
                         min_alpha = 0.007) # Menor valor possível para a Taxa de Aprendizagem

# COnstruindo o vocabulário do modelo
w2v_modelo_sg.build_vocab(lista_lista_tokens, progress_per = 100000)

# Colocando o modelo para treino
w2v_modelo_sg.train(lista_lista_tokens,
                    total_examples = w2v_modelo_sg.corpus_count,
                    epochs = 30)

2021-05-29 04:04:02,610 : - collecting all words and their counts
2021-05-29 04:04:02,612 : - PROGRESS: at sentence #0, processed 0 words, keeping 0 word types
2021-05-29 04:04:02,804 : - collected 39968 word types from a corpus of 597929 raw words and 86113 sentences
2021-05-29 04:04:02,806 : - Loading a fresh vocabulary
2021-05-29 04:04:03,033 : - effective_min_count=5 retains 13006 unique words (32% of original 39968, drops 26962)
2021-05-29 04:04:03,035 : - effective_min_count=5 leaves 552614 word corpus (92% of original 597929, drops 45315)
2021-05-29 04:04:03,103 : - deleting the raw counts dictionary of 39968 items
2021-05-29 04:04:03,106 : - sample=0.001 downsamples 11 most-common words
2021-05-29 04:04:03,108 : - downsampling leaves estimated 502900 word corpus (91.0% of prior 552614)
2021-05-29 04:04:03,169 : - estimated required memory for 13006 words and 100 dimensions: 16907800 bytes
2021-05-29 04:04:03,171 : - resetting layer weights
2021-05-29 04:04:05,830 : - training m

(15086584, 17937870)

In [ ]:
# Realizando uma comparação dos resultados de diferentes arquiteturas
w2v_modelo_sg.wv.most_similar('google')

2021-05-29 04:05:15,789 : - precomputing L2-norms of word weight vectors


[('apple', 0.6757063269615173),
 ('facebook', 0.6504735946655273),
 ('buffett', 0.6086056232452393),
 ('yahoo', 0.5912462472915649),
 ('app', 0.5536147952079773),
 ('véu', 0.5528077483177185),
 ('reguladores', 0.5432046055793762),
 ('alibaba', 0.5426156520843506),
 ('volkswagen', 0.5388311147689819),
 ('waze', 0.5383400917053223)]

In [ ]:
# Salvando os modelos para .txt

# Binary = False, para salvar os valores mesmo, sem a necessidade de estar na forma binária.
w2v_modelo.wv.save_word2vec_format("/content/drive/MyDrive/Alura/Curso_word2vec-embedding/modelo_cbow.txt", binary = False)

w2v_modelo_sg.wv.save_word2vec_format("/content/drive/MyDrive/Alura/Curso_word2vec-embedding/modelo_skipgram.txt", binary = False)

2021-05-29 04:05:51,554 : - storing 13006x300 projection weights into /content/drive/MyDrive/Alura/Curso_word2vec-embedding/modelo_cbow.txt
2021-05-29 04:05:54,624 : - storing 13006x100 projection weights into /content/drive/MyDrive/Alura/Curso_word2vec-embedding/modelo_skipgram.txt
